In [1]:
from qaravan.core import * 
from qaravan.tensorQ import *

In [2]:
# testing QubitNoise

t1 = 1000
t2 = 1000
one_qubit_time = 10
two_qubit_time = 100

nm = QubitNoise(t1, t2, one_qubit_time, two_qubit_time)
kraus_list = nm.get_kraus(time=two_qubit_time) 

sum([k.conj().T @ k for k in kraus_list])

array([[1., 0.],
       [0., 1.]])

In [12]:
# testing PauliNoise

strings = ['ii', 'ix', 'zy', 'zz']
probs = [0.9, 0.04, 0.03, 0.03]
nm = PauliNoise(strings, probs)
kraus_list = nm.get_kraus()
sum([k.conj().T @ k for k in kraus_list])

array([[1.+0.j, 0.+0.j, 0.+0.j, 0.+0.j],
       [0.+0.j, 1.+0.j, 0.+0.j, 0.+0.j],
       [0.+0.j, 0.+0.j, 1.+0.j, 0.+0.j],
       [0.+0.j, 0.+0.j, 0.+0.j, 1.+0.j]])

In [20]:
strings = ['ii', 'xx', 'yx']
probs = [0.3, 0.4, 0.3]
nm = PauliNoise(strings, probs)

nm_inv = nm.pinv()
nm_mod = nm_inv.compose(nm)
print(nm_mod)

{'ii': np.float64(1.0)}


In [21]:
ptm     = nm.ptm()                 # diagonal matrix F
ptm_inv = nm_inv.ptm()             # F^{-1}
print(np.max(np.abs(ptm_inv @ ptm - np.eye(ptm.shape[0]))))

2.220446049250313e-16


In [22]:
S      = nm.get_superop()
S_inv  = nm_inv.get_superop()
print(np.max(np.abs(S_inv @ S - np.eye(S.shape[0]))))

3.15


In [9]:
# testing PauliLindbladNoise

p1 = 0.3
p2 = 0.7

strings = ['xx', 'yx']
probs = [p1, p2]
channel = PauliLindbladNoise(strings, probs)
superop = channel.get_superop().reshape([2]*8)
print(channel.rates)

dm = random_dm(2)
state = dm.reshape([2]*4)
gate_indices, state_indices = generate_gate_indices([0,1], 2), generate_state_indices([0,1], 2)
new_state = ncon((superop, state), (gate_indices, state_indices))
new_dm = new_state.reshape(4,4)

r = channel.rates
P0, r0 = embed_operator(2, [0,1], [pauli_mapping[s] for s in strings[0]], dense=True), channel.rates[0]
P1, r1 = embed_operator(2, [0,1], [pauli_mapping[s] for s in strings[1]], dense=True), channel.rates[1]

dm_a = (1-r0)*dm + r0*P0 @ dm @ P0.T.conj()
dm_b = (1-r1)*dm_a + r1*P1 @ dm_a @ P1.T.conj()
np.allclose(dm_b, new_dm)

[np.float64(0.22559418195298675), np.float64(0.37670151802919677)]


True

In [ ]:
inv_channel = PauliLindbladNoise(strings, [-p for p in probs])
superop = inv_channel.get_superop().reshape([2]*8)
print(inv_channel.rates)

corrected_state = ncon((superop, new_state), (gate_indices, state_indices))
corrected_dm = corrected_state.reshape(4,4)
np.allclose(corrected_dm, dm)

[np.float64(-1.5275999834223373), np.float64(-0.41105940019525444)]


False

In [9]:
temp_dm = np.zeros((4,4), dtype=complex)
for k in inv_channel.get_kraus_for_string(0): 
    temp_dm += k @ new_dm @ k.T.conj()

temp_dm2 = np.zeros((4,4), dtype=complex)
for k in inv_channel.get_kraus_for_string(1):
    temp_dm2 += k @ temp_dm @ k.T.conj()

np.allclose(temp_dm2, dm)

False